# Logistic Regression with different Features and Tranfser learning

#### Modeling Features are- AMT_ANNUITY, AMT_CREDIT, EXT_SOURCE_2*EXT_SOURCE_3 and EXT_SOURCE_2/EXT_SOURCE_1
#### Evaluting features are- AMT_ANNUITY, AMT_CREDIT, DAYS_BIRTH and EXT_SOURCE_2/EXT_SOURCE_1

In [1]:
require 'open-uri'
require 'json'
require 'daru'
require 'distribution'
require 'sqlite3'
require './regression'
include Regression

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

Object

In [2]:
dir = "#{ENV['HOME']}/cs6140/final_project"
db = SQLite3::Database.new "#{dir}/credit_risk_data.db"
db.results_as_hash = true

true

In [3]:
begin db.execute("drop table cross_validation"); rescue; end
sql = <<SQL
create temp table cross_validation as 
  select sk_id_curr, abs(random()) % 10 as fold
  from application_train
  order by random()
SQL
db.execute(sql)

[]

In [4]:
sql = <<SQL
create temp view example_regression as 
  select a.sk_id_curr, 
  target,
  fold, AMT_ANNUITY, AMT_CREDIT, DAYS_BIRTH
  from application_train a inner join cross_validation c on a.sk_id_curr = c.sk_id_curr
SQL
begin db.execute("drop view example_regression"); rescue; end
db.execute(sql)

[]

In [5]:
column_types = Hash.new
db.execute("PRAGMA table_info(application_train);").each do |row|
  column_types[row["name"]] = row["type"]
end
1

1

In [6]:
def plot x, y
  Daru::DataFrame.new({x: x, y: y}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
    plot.x_label "X"
    plot.y_label "Y"
  end
end

:plot

In [7]:
class LogisticRegressionModel
  def func dataset, w
    nll = 0.0
    dataset.each {|row|
      y_predict = row["features"].keys.inject(0.0) {|total,key| total+= row["features"][key]*w[key]}
      nll += 0.5 * (y_predict - row["label"])**2 
      }
    return nll
  end
  
  def sigmoid x
    1.0 / (1 + Math.exp(-x))
  end
  
  def grad dataset, w
    g = Hash.new {0.0}
    dataset.each {|row|
      inner_product = row["features"].keys.inject(0.0) {|total,key| total+= row["features"][key]*w[key]}
      y_predict = (1/(1+Math.exp(-inner_product)))
      row["features"].keys.each{|key|
        g[key]+= (y_predict-row["label"])*row["features"][key]
        }
      }
     g.each_key {|key| 
       g[key] /= dataset.length
    }
    return g
  end
  
  def predict row, w
      y_predict = row["features"].keys.inject(0.0) {|total,key| total+= row["features"][key]*w[key]}
      y_predict = (1/(1+Math.exp(-y_predict)))
    return y_predict
  end
  
  ## Adjusts the parameter to be within the allowable range
  def adjust w
  end
end

:adjust

In [8]:
def gradient_descent zb, w, obj, learning_rate = 1e-4, rmse_tol = 1e-3, max_iter = 1000
  del_F = 1000
  last_rmse = obj.func(zb, w)
  
  iters = [0]
  rmses = [last_rmse]
  
  for i in (1..max_iter)
    if del_F < rmse_tol
      break 
    end
    if del_F > rmse_tol  
      dw = obj.grad zb, w
      w = update_weights w,dw,learning_rate
      obj.adjust w
      rmsd = obj.func zb, w
      del_F = rmses.last - rmsd
      rmses << rmsd
      iters << i
      i += 1
    end
  end
  return [iters, rmses, w, zb]
end

:gradient_descent

In [9]:
def roc_curve(scores)
  total_neg = scores.inject(0.0) {|u,s| u += (1 - s.last)}
  total_pos = scores.inject(0.0) {|u,s| u += s.last}
  c_neg = 0.0
  c_pos = 0.0
  fp = [0.0]
  tp = [0.0]
  auc = 0.0
  scores.sort_by {|s| -s.first}.each do |s|
    c_neg += 1 if s.last <= 0
    c_pos += 1 if s.last > 0  

    fpr = c_neg / total_neg
    tpr = c_pos / total_pos
    auc += 0.5 * (tpr + tp.last) * (fpr - fp.last)
    fp << fpr
    tp << tpr
  end
  return [fp, tp, auc]
end

:roc_curve

In [23]:
batch_data = db.execute("SELECT SK_ID_CURR, TARGET, AMT_ANNUITY, AMT_CREDIT, EXT_SOURCE_2*EXT_SOURCE_3, EXT_SOURCE_2/EXT_SOURCE_1 FROM application_train limit 10000;")
data = labelData batch_data , column_types
1

1

In [25]:
iters, rmses, w, zb = gradient_descent data, Hash.new {|h,k| h[k] = 0.5} , LogisticRegressionModel.new , 1e-8, 1e-3, 1000
plot iters, rmses

#<Nyaplot::Plot:0x007fe6e4510838 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fe6e1177918 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"1bb92964-8190-477e-9b8c-0e39d2a93329"}, @xrange=[0, 104], @yrange=[14233679726024.736, 3551188269622452.5]>], :options=>{:x_label=>"X", :y_label=>"Y", :zoom=>true, :width=>700, :xrange=>[0, 104], :yrange=>[14233679726024.736, 3551188269622452.5]}}>

### Logistic Regression
*Graph between Iterations and RMSE*
![xfeatures](feature_rmse.png)


### Using experimental Features to train the model and using perdicting using other values

In [26]:
sql = "SELECT SK_ID_CURR, TARGET, AMT_ANNUITY, AMT_CREDIT, DAYS_BIRTH, EXT_SOURCE_2/EXT_SOURCE_1 FROM application_train limit 5000;"
scores = score_model LogisticRegressionModel.new, db, w, sql
f_p, t_p, auc = roc_curve scores
puts auc
plot f_p, t_p

0.56701164062517


#<Nyaplot::Plot:0x007fe6e198ea48 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fe6e1e39340 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"2520b321-f8f5-4053-aa6e-098fce59d804"}, @xrange=[0.0, 1.0], @yrange=[0.0, 1.0]>], :options=>{:x_label=>"X", :y_label=>"Y", :zoom=>true, :width=>700, :xrange=>[0.0, 1.0], :yrange=>[0.0, 1.0]}}>

### Logistic Regression
*ROC Curve*
<br>Accuracy is 0.56701164062517
![xfeatures](feature_roc.png)